In [ ]:
import os, ast, json, itertools, random, joblib, warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from joblib import Parallel, delayed

from sklearn.tree           import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing   import StandardScaler
from sklearn.pipeline        import make_pipeline
from sklearn.metrics         import accuracy_score

warnings.filterwarnings("ignore")

# ---------------------------
# 0-A. Функції завантаження
# ---------------------------
def load_tracks(filepath):
    tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])
    _expand_string_lists(tracks)
    _convert_date_columns(tracks)
    _convert_subsets(tracks)
    _convert_to_category(tracks)
    return tracks

def _expand_string_lists(tracks_df):
    list_cols = [('track','tags'),('album','tags'),('artist','tags'),
                 ('track','genres'),('track','genres_all')]
    for c in list_cols:
        tracks_df[c] = tracks_df[c].map(ast.literal_eval)

def _convert_date_columns(tracks_df):
    date_cols = [
        ('track','date_created'),('track','date_recorded'),
        ('album','date_created'),('album','date_released'),
        ('artist','date_created'),('artist','active_year_begin'),
        ('artist','active_year_end')
    ]
    for c in date_cols:
        tracks_df[c] = pd.to_datetime(tracks_df[c])

def _convert_subsets(tracks_df):
    from pandas.api.types import CategoricalDtype
    tracks_df[('set','subset')] = tracks_df[('set','subset')].astype(
        CategoricalDtype(categories=('small','medium','large'), ordered=True)
    )

def _convert_to_category(tracks_df):
    cat_cols = [('track','genre_top'),('track','license'),
                ('album','type'),('album','information'),('artist','bio')]
    for c in cat_cols:
        tracks_df[c] = tracks_df[c].astype('category')

def load_features(filepath):
    df = pd.read_csv(filepath, index_col=0, header=[0,1,2])
    df.columns = [f"{a}_{b}_{c}" for a,b,c in df.columns]
    return df

# ---------------------------
# 0-B. Завантаження FMA-даних
# ---------------------------
BASE_DIR     = r"C:/Users/Climclaff/Downloads/fma/fma_metadata"
tracks_path  = os.path.join(BASE_DIR, "tracks.csv")
features_path= os.path.join(BASE_DIR, "features.csv")

tracks   = load_tracks(tracks_path)
features = load_features(features_path)

# маска «fma_small» та виключені Pop / Folk
mask_small   = tracks['set','subset'].eq('small')
mask_genre   = ~tracks['track','genre_top'].isin(["Pop","Folk"])
filtered_idx = mask_small & mask_genre

# групи ознак
all_cols = features.columns
FEATURE_GROUPS = {
    "chroma_cens":        [c for c in all_cols if "chroma_cens"        in c],
    "chroma_cqt":         [c for c in all_cols if "chroma_cqt"         in c],
    "chroma_stft":        [c for c in all_cols if "chroma_stft"        in c],
    "mfcc":               [c for c in all_cols if "mfcc"               in c],
    "rmse":               [c for c in all_cols if "rmse"               in c],
    "spectral_bandwidth": [c for c in all_cols if "spectral_bandwidth" in c],
    "spectral_centroid":  [c for c in all_cols if "spectral_centroid"  in c],
    "spectral_contrast":  [c for c in all_cols if "spectral_contrast"  in c],
    "spectral_rolloff":   [c for c in all_cols if "spectral_rolloff"   in c],
    "tonnetz":            [c for c in all_cols if "tonnetz"            in c],
    "zcr":                [c for c in all_cols if "zcr"                in c],
}

y_all = tracks.loc[filtered_idx, ('track','genre_top')].to_numpy()

In [ ]:
def evaluate_combo(col_list, X_full, y_full, sample_size, random_state):
    idx = (np.arange(len(y_full)))
    trn, tst = train_test_split(
        idx, train_size=sample_size, stratify=y_full, random_state=random_state
    )
    X_train, X_test = X_full.iloc[trn], X_full.iloc[tst]
    y_train, y_test = y_full[trn], y_full[tst]

    clf = DecisionTreeClassifier(random_state=random_state, max_depth=None)
    clf.fit(X_train[col_list], y_train)
    return accuracy_score(y_test, clf.predict(X_test[col_list]))

def greedy_search_feature_groups(
        feature_groups: dict,
        X_full: pd.DataFrame,
        y_full: np.ndarray,
        sample_size: int = 1000,
        max_comb_size: int = 8,
        max_evals: int = 5000,
        n_jobs: int = -1,
        random_state: int = 42):
    rng = np.random.RandomState(random_state)

    # Попередньо створюємо список усіх комбінацій 2..max_comb_size
    all_combos = []
    for k in range(2, max_comb_size+1):
        all_combos.extend(itertools.combinations(feature_groups.keys(), k))
    rng.shuffle(all_combos)                 # випадковий порядок
    all_combos = all_combos[:max_evals]     # обмежуємо кількість оцінок

    def _eval(combo):
        cols = sum((feature_groups[g] for g in combo), [])
        score = evaluate_combo(cols, X_full, y_full, sample_size, random_state)
        return combo, score

    results = Parallel(n_jobs=n_jobs)(
        delayed(_eval)(cmb) for cmb in tqdm(all_combos, desc="Searching")
    )

    best_combo, best_score = max(results, key=lambda x: x[1])
    return best_combo, best_score, results

# --- запуск ---
SAMPLE_SIZE   = 1200     
MAX_COMB_SIZE = 11
MAX_EVALS     = 10000

best_combo, best_acc, search_log = greedy_search_feature_groups(
    FEATURE_GROUPS,
    features.loc[filtered_idx],
    y_all,
    sample_size   = SAMPLE_SIZE,
    max_comb_size = MAX_COMB_SIZE,
    max_evals     = MAX_EVALS,
    n_jobs        = -1,
    random_state  = 123
)

print(f"\n★ Best combo: {best_combo}  | Accuracy={best_acc:.4f}")

In [ ]:
BEST_GROUPS = best_combo  
selected_cols = sum((FEATURE_GROUPS[g] for g in BEST_GROUPS), [])

X_all = features.loc[filtered_idx, selected_cols].to_numpy()

print("Shape after feature-selection:", X_all.shape)

In [ ]:
def make_svc(C=1.0, gamma='scale', kernel='rbf'):
    svc = make_pipeline(
        StandardScaler(),
        SVC(C=C, gamma=gamma, kernel=kernel, probability=True, random_state=213)
    )
    return svc

def make_bagging(
        n_estimators=100, max_samples=1.0, max_features=1.0,
        max_depth=None, min_samples_split=2, bootstrap=True):
    base_dt = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=213
    )
    return BaggingClassifier(
        estimator      = base_dt,
        n_estimators   = n_estimators,
        max_samples    = max_samples,
        max_features   = max_features,
        bootstrap      = bootstrap,
        n_jobs         = -1,
        random_state   = 213
    )

def make_adaboost(
        n_estimators=200, learning_rate=0.1,
        max_depth=3, min_samples_split=2, min_samples_leaf=1):
    base_dt = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=213
    )
    return AdaBoostClassifier(
        estimator     = base_dt,
        n_estimators  = n_estimators,
        learning_rate = learning_rate,
        random_state  = 213
    )

def make_random_forest(
        n_estimators=300, max_depth=None, min_samples_split=2,
        min_samples_leaf=1, max_features='sqrt', bootstrap=True):
    return RandomForestClassifier(
        n_estimators      = n_estimators,
        max_depth         = max_depth,
        min_samples_split = min_samples_split,
        min_samples_leaf  = min_samples_leaf,
        max_features      = max_features,
        bootstrap         = bootstrap,
        n_jobs            = -1,
        random_state      = 213
    )

In [ ]:
import optuna
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, valid_idx = next(sss.split(X_all, y_all))
X_train, y_train = X_all[train_idx], y_all[train_idx]
X_valid, y_valid = X_all[valid_idx], y_all[valid_idx]

# ---------------------------
# 4-B. Оптимізація SVC
# ---------------------------
def objective_svc(trial):
    C     = trial.suggest_float("C",     1e-3, 1e3, log=True)
    gamma = trial.suggest_float("gamma", 1e-4, 1e1, log=True)
    kernel= 'rbf'

    model = make_svc(C=C, gamma=gamma, kernel=kernel)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)

N_TRIALS_SVC = 30          
study_svc = optuna.create_study(direction="maximize")
study_svc.optimize(objective_svc, n_trials=N_TRIALS_SVC)

BEST_SVC_PARAMS = study_svc.best_params
print("Best SVC params:", BEST_SVC_PARAMS,
      f"| val-acc = {study_svc.best_value:.4f}")

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

def objective_bagging(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 400)
    max_samples  = trial.suggest_float("max_samples", 0.2, 1.0)
    max_features = trial.suggest_float("max_features", 0.2, 1.0)
    max_depth    = trial.suggest_int("max_depth", 4, 20)
    min_split    = trial.suggest_int("min_split", 2, 30)
    bootstrap    = trial.suggest_categorical("bootstrap", [True, False])

    model = make_bagging(
        n_estimators=n_estimators,
        max_samples=max_samples,
        max_features=max_features,
        max_depth=max_depth,
        min_samples_split=min_split,
        bootstrap=bootstrap
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)

N_TRIALS_BAG = 30
study_bag = optuna.create_study(direction="maximize")
study_bag.optimize(objective_bagging, n_trials=N_TRIALS_BAG)

BEST_BAGGING_PARAMS = study_bag.best_params
print("Best Bagging params:", BEST_BAGGING_PARAMS,
      f"| val-acc = {study_bag.best_value:.4f}")

In [ ]:
def objective_ada(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    lr           = trial.suggest_float("learning_rate", 1e-3, 1.0, log=True)
    max_depth    = trial.suggest_int("max_depth", 2, 12)
    min_split    = trial.suggest_int("min_split", 2, 30)
    min_leaf     = trial.suggest_int("min_leaf", 1, 20)

    model = make_adaboost(
        n_estimators=n_estimators,
        learning_rate=lr,
        max_depth=max_depth,
        min_samples_split=min_split,
        min_samples_leaf=min_leaf
    )
    model.fit(X_train, y_train)
    return accuracy_score(y_valid, model.predict(X_valid))

study_ada = optuna.create_study(direction="maximize")
study_ada.optimize(objective_ada, n_trials=30)
BEST_ADA_PARAMS = study_ada.best_params
print("Best AdaBoost params:", BEST_ADA_PARAMS,
      f"| val-acc = {study_ada.best_value:.4f}")

In [ ]:
def objective_rf(trial):
    n_estimators = trial.suggest_int("n_estimators", 100, 600)
    max_depth    = trial.suggest_int("max_depth", 4, 30)
    min_split    = trial.suggest_int("min_split", 2, 20)
    min_leaf     = trial.suggest_int("min_leaf", 1, 10)
    max_features = trial.suggest_float("max_features", 0.2, 1.0)
    bootstrap    = trial.suggest_categorical("bootstrap", [True, False])

    model = make_random_forest(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_split,
        min_samples_leaf=min_leaf,
        max_features=max_features,
        bootstrap=bootstrap
    )
    model.fit(X_train, y_train)
    return accuracy_score(y_valid, model.predict(X_valid))

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_rf, n_trials=30)
BEST_RF_PARAMS = study_rf.best_params
print("Best Random Forest params:", BEST_RF_PARAMS,
      f"| val-acc = {study_rf.best_value:.4f}")